Étape 1 – Configuration des variables d'environnement

In [9]:
# Paramètres
TAVILY_API_KEY   = "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"
MCP_HTTP_TOKEN   = "mcp-token-123"
OLLAMA_URL       = "http://localhost:11434"
SERVER_PORT      = 5000
NGROK_TOKEN      = "30Y4I20WWMblLpoQmZBTJyvdfis_4tfap1sfjHNepzEv9yrdk"

Étape 2 – Créer l'arborescence

In [10]:
import os
os.makedirs("/content/mcp-brief-bot/examples", exist_ok=True)

Étape 3 – Installer les dépendances

In [11]:
!pip install flask tavily-python trafilatura python-dotenv pyngrok requests -q

Étape 4 – Server.py corrigé

In [12]:
%%writefile /content/mcp-brief-bot/server.py
import os
import json
import threading
import time
import requests
from flask import Flask, request, jsonify
from tavily import TavilyClient
from trafilatura import extract
from dotenv import load_dotenv

load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "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")
MCP_HTTP_TOKEN = os.getenv("MCP_HTTP_TOKEN", "mcp-token-123")
SERVER_PORT = int(os.getenv("SERVER_PORT", 5000))

tavily = TavilyClient(api_key=TAVILY_API_KEY)
app = Flask(__name__)

def auth_required(f):
    def wrapper(*args, **kwargs):
        auth_header = request.headers.get("Authorization")
        if not auth_header or auth_header != f"Bearer {MCP_HTTP_TOKEN}":
            return jsonify({"error": "Unauthorized"}), 401
        return f(*args, **kwargs)
    wrapper.__name__ = f.__name__
    return wrapper

@app.route("/tools", methods=["GET"])
@auth_required
def list_tools():
    return jsonify([
        {
            "name": "search_web",
            "description": "Search the web for information",
            "parameters": {
                "query": {"type": "string", "description": "Search query"},
                "k": {"type": "number", "default": 3, "description": "Number of results"}
            }
        },
        {
            "name": "fetch_readable",
            "description": "Extract readable text from URL",
            "parameters": {
                "url": {"type": "string", "description": "URL to extract text from"}
            }
        },
        {
            "name": "summarize_with_citations",
            "description": "Summarize documents with citations",
            "parameters": {
                "topic": {"type": "string", "description": "Topic to summarize"},
                "docs": {"type": "array", "description": "Array of documents"}
            }
        },
        {
            "name": "save_markdown",
            "description": "Save content as markdown file",
            "parameters": {
                "filename": {"type": "string", "description": "Filename"},
                "content": {"type": "string", "description": "Markdown content"}
            }
        }
    ])

@app.route("/tools/search_web", methods=["POST"])
@auth_required
def search_web():
    try:
        data = request.get_json()
        if not data or "query" not in data:
            return jsonify({"error": "Query parameter required"}), 400

        query = data["query"]
        max_results = data.get("k", 3)

        results = tavily.search(query=query, max_results=max_results)

        formatted_results = []
        for r in results.get("results", []):
            formatted_results.append({
                "title": r.get("title", ""),
                "url": r.get("url", ""),
                "snippet": r.get("snippet", ""),
                "source": "tavily"
            })

        return jsonify(formatted_results)
    except Exception as e:
        return jsonify({"error": f"Search failed: {str(e)}"}), 500

@app.route("/tools/fetch_readable", methods=["POST"])
@auth_required
def fetch_readable():
    try:
        data = request.get_json()
        if not data or "url" not in data:
            return jsonify({"error": "URL parameter required"}), 400

        url = data["url"]

        headers = {
            "User-Agent": "MCP-Brief-Bot/1.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8"
        }

        response = requests.get(url, timeout=15, headers=headers)
        response.raise_for_status()

        text = extract(response.text, url=url, include_comments=False, include_tables=False)

        if not text:
            return jsonify({"error": "Could not extract text from URL"}), 400

        return jsonify({
            "url": url,
            "title": text or "",
            "text": text or ""
        })
    except requests.exceptions.RequestException as e:
        return jsonify({"error": f"Failed to fetch URL: {str(e)}"}), 400
    except Exception as e:
        return jsonify({"error": f"Text extraction failed: {str(e)}"}), 500

@app.route("/tools/summarize_with_citations", methods=["POST"])
@auth_required
def summarize_with_citations():
    try:
        data = request.get_json()
        if not data or "docs" not in data:
            return jsonify({"error": "Docs parameter required"}), 400

        docs = data["docs"]
        topic = data.get("topic", "")

        bullets = []
        for idx, doc in enumerate(docs, 1):
            text = doc.get("text", "")
            if text:
                # Diviser en phrases et prendre les 3 premières
                sentences = [s.strip() for s in text.split(".") if s.strip()]
                summary_text = ". ".join(sentences[:3])
                if summary_text and not summary_text.endswith("."):
                    summary_text += "."
                bullets.append(f"{summary_text} [{idx}]")
            else:
                bullets.append(f"Contenu non disponible. [{idx}]")

        sources = []
        for i, doc in enumerate(docs):
            sources.append({
                "i": i + 1,
                "title": doc.get("title", "Sans titre"),
                "url": doc.get("url", "")
            })

        return jsonify({
            "bullets": bullets,
            "sources": sources
        })
    except Exception as e:
        return jsonify({"error": f"Summarization failed: {str(e)}"}), 500

@app.route("/tools/save_markdown", methods=["POST"])
@auth_required
def save_markdown():
    try:
        data = request.get_json()
        if not data or "filename" not in data or "content" not in data:
            return jsonify({"error": "Filename and content parameters required"}), 400

        filename = data["filename"]
        content = data["content"]

        # Assurer que le répertoire existe
        os.makedirs("/content/mcp-brief-bot/examples", exist_ok=True)

        path = f"/content/mcp-brief-bot/examples/{filename}"

        with open(path, "w", encoding="utf-8") as f:
            f.write(content)

        return jsonify({"path": path, "message": "File saved successfully"})
    except Exception as e:
        return jsonify({"error": f"Failed to save file: {str(e)}"}), 500

@app.route("/health", methods=["GET"])
def health_check():
    return jsonify({"status": "healthy", "timestamp": time.time()})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=SERVER_PORT, debug=False)

Overwriting /content/mcp-brief-bot/server.py


Étape 5 – Lancement du serveur avec ngrok (corrigé)

In [13]:
from pyngrok import ngrok
import threading
import os
import subprocess
import time
import requests

# Configuration ngrok
ngrok.set_auth_token("30Y4I20WWMblLpoQmZBTJyvdfis_4tfap1sfjHNepzEv9yrdk")

# Démarrage du tunnel
tunnel = ngrok.connect(5000)
PUBLIC_URL = tunnel.public_url
print(f"🌐 URL publique : {PUBLIC_URL}")

def start_server():
    try:
        os.chdir("/content/mcp-brief-bot")
        subprocess.run(["python", "-u", "server.py"], check=True)
    except Exception as e:
        print(f"Erreur serveur: {e}")

# Démarrage du serveur en arrière-plan
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# Attendre que le serveur démarre
print("⏳ Démarrage du serveur...")
time.sleep(5)

# Test de santé
try:
    health_response = requests.get(f"http://localhost:5000/health", timeout=10)
    if health_response.status_code == 200:
        print("✅ Serveur démarré avec succès")
    else:
        print("⚠️ Serveur démarré mais problème de santé")
except Exception as e:
    print(f"❌ Erreur de connexion au serveur: {e}")

# Test de l'API
try:
    test_response = requests.post(
        "http://localhost:5000/tools/search_web",
        headers={"Authorization": "Bearer mcp-token-123"},
        json={"query": "AI 2025", "k": 2},
        timeout=15
    )
    print(f"📊 Test API - Status: {test_response.status_code}")
    if test_response.status_code == 200:
        print("✅ API fonctionnelle")
        print(f"📝 Résultat: {str(test_response.json())[:200]}...")
    else:
        print(f"❌ Erreur API: {test_response.text}")
except Exception as e:
    print(f"❌ Erreur test API: {e}")

print(f"\n🎯 URL publique pour tests externes: {PUBLIC_URL}")

# Diagnostic complet
def run_diagnostics():
    print("\n🔍 DIAGNOSTIC COMPLET")
    print("="*50)

    # 1. Vérifier les variables
    try:
        print(f"✅ PUBLIC_URL: {PUBLIC_URL}")
        print(f"✅ MCP_HTTP_TOKEN: {MCP_HTTP_TOKEN}")
    except NameError as e:
        print(f"❌ Variable manquante: {e}")
        return False

    # 2. Test de connexion locale
    try:
        local_health = requests.get("http://localhost:5000/health", timeout=5)
        print(f"✅ Serveur local: {local_health.status_code}")
    except Exception as e:
        print(f"❌ Serveur local inaccessible: {e}")
        return False

    # 3. Test de connexion publique
    try:
        public_health = requests.get(f"{PUBLIC_URL}/health", timeout=10)
        print(f"✅ Serveur public: {public_health.status_code}")
    except Exception as e:
        print(f"❌ Serveur public inaccessible: {e}")
        print("   💡 Vérifiez que ngrok est bien configuré")
        return False

    # 4. Test d'authentification
    try:
        auth_test = requests.get(
            f"{PUBLIC_URL}/tools",
            headers={"Authorization": f"Bearer {MCP_HTTP_TOKEN}"},
            timeout=10
        )
        print(f"✅ Authentification: {auth_test.status_code}")
    except Exception as e:
        print(f"❌ Problème d'authentification: {e}")
        return False

    print("="*50)
    print("🎉 Tous les tests passent ! Le serveur est prêt.")
    return True

# Lancer le diagnostic
run_diagnostics()

🌐 URL publique : https://886f512c8dc5.ngrok-free.app
⏳ Démarrage du serveur...
Erreur serveur: Command '['python', '-u', 'server.py']' returned non-zero exit status 1.
✅ Serveur démarré avec succès
📊 Test API - Status: 200
✅ API fonctionnelle
📝 Résultat: [{'snippet': '', 'source': 'tavily', 'title': 'The 2025 AI Index Report | Stanford HAI', 'url': 'https://hai.stanford.edu/ai-index/2025-ai-index-report'}, {'snippet': '', 'source': 'tavily', 'title': ...

🎯 URL publique pour tests externes: https://886f512c8dc5.ngrok-free.app

🔍 DIAGNOSTIC COMPLET
✅ PUBLIC_URL: https://886f512c8dc5.ngrok-free.app
✅ MCP_HTTP_TOKEN: mcp-token-123
✅ Serveur local: 200
✅ Serveur public: 200
✅ Authentification: 200
🎉 Tous les tests passent ! Le serveur est prêt.


True

Étape 6 – Client.py corrigé

In [14]:
%%writefile /content/mcp-brief-bot/client.py
import sys
import json
import datetime
import requests
import os
from urllib.parse import urlparse

# Configuration
TOKEN = os.getenv("MCP_HTTP_TOKEN", "mcp-token-123")

def get_base_url(url_arg):
    """Extraire l'URL de base à partir de l'argument"""
    if "://" not in url_arg:
        url_arg = "https://" + url_arg
    parsed = urlparse(url_arg)
    return f"{parsed.scheme}://{parsed.netloc}"

def make_request(base_url, route, data):
    """Effectuer une requête POST vers l'API"""
    try:
        url = f"{base_url}/tools/{route}"
        headers = {
            "Authorization": f"Bearer {TOKEN}",
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, json=data, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de la requête {route}: {e}")
        if hasattr(e, 'response') and e.response:
            print(f"   Détails: {e.response.text}")
        sys.exit(1)

def main():
    if len(sys.argv) < 2:
        print("Usage: python client.py <URL_BASE> [TOPIC]")
        print("Exemple: python client.py https://abc123.ngrok.io 'quantum computing 2025'")
        sys.exit(1)

    BASE_URL = get_base_url(sys.argv[1])
    TOPIC = " ".join(sys.argv[2:]) if len(sys.argv) > 2 else "AI news"

    print(f"🔍 Recherche sur le sujet: {TOPIC}")
    print(f"🌐 URL de base: {BASE_URL}")

    # 1. Recherche web
    print("\n📡 Étape 1: Recherche web...")
    search_results = make_request(BASE_URL, "search_web", {"query": TOPIC, "k": 5})
    print(f"   Trouvé {len(search_results)} résultats")

    # 2. Extraction du contenu des 3 premiers résultats
    print("\n📄 Étape 2: Extraction du contenu...")
    docs = []
    for i, result in enumerate(search_results[:3], 1):
        print(f"   Extraction {i}/3: {result['title'][:50]}...")
        try:
            content = make_request(BASE_URL, "fetch_readable", {"url": result["url"]})
            docs.append({
                "title": content.get("title", result["title"]),
                "url": result["url"],
                "text": content.get("text", "")
            })
        except Exception as e:
            print(f"   ⚠️ Échec extraction pour {result['url']}: {e}")
            docs.append({
                "title": result["title"],
                "url": result["url"],
                "text": result.get("snippet", "")
            })

    # 3. Résumé avec citations
    print("\n✍️ Étape 3: Génération du résumé...")
    summary = make_request(BASE_URL, "summarize_with_citations", {
        "topic": TOPIC,
        "docs": docs
    })

    # 4. Génération du markdown
    print("\n📝 Étape 4: Génération du rapport...")
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")

    markdown_content = f"""# Brief: {TOPIC}
*Généré le {datetime.datetime.now().strftime("%d/%m/%Y à %H:%M")}*

## Résumé

"""

    for bullet in summary["bullets"]:
        markdown_content += f"- {bullet}\n"

    markdown_content += "\n## Sources\n\n"

    for source in summary["sources"]:
        markdown_content += f"{source['i']}. [{source['title']}]({source['url']})\n"

    # 5. Sauvegarde
    filename = f"brief_{TOPIC.replace(' ', '_')}_{timestamp}.md"
    save_result = make_request(BASE_URL, "save_markdown", {
        "filename": filename,
        "content": markdown_content
    })

    print(f"✅ Rapport sauvegardé: {save_result['path']}")
    print(f"\n📊 Résumé généré avec {len(summary['bullets'])} points et {len(summary['sources'])} sources")

if __name__ == "__main__":
    main()

Overwriting /content/mcp-brief-bot/client.py


Étape 8 : Validation

In [16]:
# =============================================================================
# SOLUTION COMPLÈTE POUR DEBUGGER ET CORRIGER LE PROBLÈME
# =============================================================================

import os
import subprocess
import requests
import threading
import time
from pyngrok import ngrok

# 1. VÉRIFICATION DE L'ÉTAT ACTUEL
print("🔍 DIAGNOSTIC DE L'ÉTAT ACTUEL")
print("="*60)

# Vérifier les variables d'environnement
print("📋 Variables actuelles:")
try:
    print(f"   PUBLIC_URL: {PUBLIC_URL}")
except NameError:
    print("   ❌ PUBLIC_URL n'est pas définie")
    PUBLIC_URL = None

# Vérifier les processus en cours
try:
    result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
    flask_processes = [line for line in result.stdout.split('\n') if 'server.py' in line]
    ngrok_processes = [line for line in result.stdout.split('\n') if 'ngrok' in line]

    print(f"   Processus Flask: {len(flask_processes)} trouvé(s)")
    print(f"   Processus ngrok: {len(ngrok_processes)} trouvé(s)")
except:
    print("   ⚠️ Impossible de vérifier les processus")

print("\n" + "="*60)

# 2. REDÉMARRAGE COMPLET
print("🔄 REDÉMARRAGE COMPLET DU SERVICE")
print("="*60)

# Arrêter tous les tunnels ngrok existants
try:
    ngrok.disconnect()
    ngrok.kill()
    print("✅ Tunnels ngrok fermés")
except:
    print("⚠️ Pas de tunnels ngrok à fermer")

# Tuer les processus Flask existants
try:
    subprocess.run(['pkill', '-f', 'server.py'], check=False)
    print("✅ Processus Flask arrêtés")
except:
    print("⚠️ Pas de processus Flask à arrêter")

time.sleep(2)

# 3. CONFIGURATION NGROK
print("\n📡 Configuration ngrok...")
try:
    ngrok.set_auth_token("30Y4I20WWMblLpoQmZBTJyvdfis_4tfap1sfjHNepzEv9yrdk")
    print("✅ Token ngrok configuré")
except Exception as e:
    print(f"❌ Erreur configuration ngrok: {e}")

# 4. DÉMARRAGE DU SERVEUR
print("\n🚀 Démarrage du serveur Flask...")

def start_flask_server():
    try:
        os.chdir("/content/mcp-brief-bot")
        # Utiliser exec pour remplacer le processus
        os.system("python server.py &")
    except Exception as e:
        print(f"❌ Erreur serveur: {e}")

# Démarrer le serveur en arrière-plan
server_thread = threading.Thread(target=start_flask_server, daemon=True)
server_thread.start()

# Attendre le démarrage
print("⏳ Attente du démarrage du serveur...")
for i in range(10):
    try:
        response = requests.get("http://localhost:5000/health", timeout=2)
        if response.status_code == 200:
            print(f"✅ Serveur démarré (tentative {i+1})")
            break
    except:
        time.sleep(1)
        print(f"   Tentative {i+1}/10...")
else:
    print("❌ Impossible de démarrer le serveur local")
    exit(1)

# 5. CRÉATION DU TUNNEL NGROK
print("\n🌐 Création du tunnel ngrok...")
try:
    tunnel = ngrok.connect(5000)
    PUBLIC_URL = tunnel.public_url
    print(f"✅ Tunnel créé: {PUBLIC_URL}")
except Exception as e:
    print(f"❌ Erreur tunnel ngrok: {e}")
    exit(1)

# 6. TESTS DE VALIDATION
print("\n🧪 Tests de validation...")

def test_endpoint(url, description):
    try:
        response = requests.get(url, timeout=10)
        print(f"✅ {description}: {response.status_code}")
        return True
    except Exception as e:
        print(f"❌ {description}: {e}")
        return False

# Test local
test_endpoint("http://localhost:5000/health", "Serveur local")

# Test public
test_endpoint(f"{PUBLIC_URL}/health", "Serveur public")

# Test API avec authentification
try:
    response = requests.post(
        f"{PUBLIC_URL}/tools/search_web",
        headers={"Authorization": "Bearer mcp-token-123"},
        json={"query": "AI news", "k": 2},
        timeout=30
    )
    if response.status_code == 200:
        results = response.json()
        print(f"✅ API Search: {len(results)} résultats")
    else:
        print(f"❌ API Search: {response.status_code}")
except Exception as e:
    print(f"❌ API Search: {e}")

print("\n" + "="*60)
print("🎯 INFORMATIONS FINALES")
print(f"URL publique: {PUBLIC_URL}")
print(f"Token: mcp-token-123")
print("="*60)

# 7. CLIENT DE TEST INTÉGRÉ
print("\n🔧 Test du client intégré...")

def test_client_integrated():
    try:
        # 1. Search
        print("📡 Recherche web...")
        search_response = requests.post(
            f"{PUBLIC_URL}/tools/search_web",
            headers={"Authorization": "Bearer mcp-token-123"},
            json={"query": "quantum computing 2025", "k": 3},
            timeout=30
        )
        search_response.raise_for_status()
        search_results = search_response.json()
        print(f"   ✅ {len(search_results)} résultats trouvés")

        # 2. Fetch content
        print("📄 Extraction du contenu...")
        docs = []
        for i, result in enumerate(search_results[:2], 1):
            try:
                content_response = requests.post(
                    f"{PUBLIC_URL}/tools/fetch_readable",
                    headers={"Authorization": "Bearer mcp-token-123"},
                    json={"url": result["url"]},
                    timeout=15
                )
                if content_response.status_code == 200:
                    content = content_response.json()
                    docs.append({
                        "title": content.get("title", result["title"]),
                        "url": result["url"],
                        "text": content.get("text", result.get("snippet", ""))
                    })
                    print(f"   ✅ Contenu extrait {i}/2")
                else:
                    docs.append({
                        "title": result["title"],
                        "url": result["url"],
                        "text": result.get("snippet", "")
                    })
                    print(f"   ⚠️ Utilisation du snippet {i}/2")
            except:
                docs.append({
                    "title": result["title"],
                    "url": result["url"],
                    "text": result.get("snippet", "")
                })
                print(f"   ⚠️ Erreur extraction {i}/2, utilisation du snippet")

        # 3. Summarize
        print("✍️ Génération du résumé...")
        summary_response = requests.post(
            f"{PUBLIC_URL}/tools/summarize_with_citations",
            headers={"Authorization": "Bearer mcp-token-123"},
            json={"topic": "quantum computing 2025", "docs": docs},
            timeout=15
        )
        summary_response.raise_for_status()
        summary = summary_response.json()
        print(f"   ✅ Résumé généré avec {len(summary['bullets'])} points")

        # 4. Save
        import datetime
        markdown_content = f"""# Brief: Quantum Computing 2025
*Généré le {datetime.datetime.now().strftime("%d/%m/%Y à %H:%M")}*

## Résumé

"""
        for bullet in summary["bullets"]:
            markdown_content += f"- {bullet}\n"

        markdown_content += "\n## Sources\n\n"
        for source in summary["sources"]:
            markdown_content += f"{source['i']}. [{source['title']}]({source['url']})\n"

        save_response = requests.post(
            f"{PUBLIC_URL}/tools/save_markdown",
            headers={"Authorization": "Bearer mcp-token-123"},
            json={
                "filename": f"brief_quantum_computing_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}.md",
                "content": markdown_content
            },
            timeout=10
        )
        save_response.raise_for_status()
        save_result = save_response.json()
        print(f"   ✅ Fichier sauvegardé: {save_result['path']}")

        return True

    except Exception as e:
        print(f"❌ Erreur test client: {e}")
        return False

# Lancer le test intégré
success = test_client_integrated()

if success:
    print("\n🎉 SUCCÈS ! Le système fonctionne parfaitement.")
    print("\n📝 Vous pouvez maintenant utiliser le client externe:")
    print(f"   python /content/mcp-brief-bot/client.py {PUBLIC_URL} \"votre sujet\"")
else:
    print("\n❌ Des problèmes persistent. Vérifiez les logs ci-dessus.")

print("\n" + "="*60)

🔍 DIAGNOSTIC DE L'ÉTAT ACTUEL
📋 Variables actuelles:
   PUBLIC_URL: https://886f512c8dc5.ngrok-free.app
   Processus Flask: 1 trouvé(s)
   Processus ngrok: 1 trouvé(s)

🔄 REDÉMARRAGE COMPLET DU SERVICE
⚠️ Pas de tunnels ngrok à fermer
✅ Processus Flask arrêtés
Erreur serveur: Command '['python', '-u', 'server.py']' died with <Signals.SIGTERM: 15>.

📡 Configuration ngrok...
✅ Token ngrok configuré

🚀 Démarrage du serveur Flask...
⏳ Attente du démarrage du serveur...
   Tentative 1/10...
✅ Serveur démarré (tentative 2)

🌐 Création du tunnel ngrok...
✅ Tunnel créé: https://fd716c056990.ngrok-free.app

🧪 Tests de validation...
✅ Serveur local: 200
✅ Serveur public: 200
✅ API Search: 2 résultats

🎯 INFORMATIONS FINALES
URL publique: https://fd716c056990.ngrok-free.app
Token: mcp-token-123

🔧 Test du client intégré...
📡 Recherche web...
   ✅ 3 résultats trouvés
📄 Extraction du contenu...
   ⚠️ Utilisation du snippet 1/2
   ✅ Contenu extrait 2/2
✍️ Génération du résumé...
   ✅ Résumé généré a

Etape 9 : Test

In [17]:
# Test direct avec votre URL active
PUBLIC_URL = "https://fd716c056990.ngrok-free.app"

import subprocess
result = subprocess.run([
    "python", "/content/mcp-brief-bot/client.py",
    PUBLIC_URL, "robotique médicale"
], capture_output=True, text=True)

print("SORTIE:")
print(result.stdout)
if result.stderr:
    print("ERREURS:")
    print(result.stderr)

SORTIE:
🔍 Recherche sur le sujet: robotique médicale
🌐 URL de base: https://fd716c056990.ngrok-free.app

📡 Étape 1: Recherche web...
   Trouvé 5 résultats

📄 Étape 2: Extraction du contenu...
   Extraction 1/3: Innovations en robotique médicale : révolutionner ...
   Extraction 2/3: Robot médical - Wikipédia...
   Extraction 3/3: La robotique en médecine - KUKA Robotics...

✍️ Étape 3: Génération du résumé...

📝 Étape 4: Génération du rapport...
✅ Rapport sauvegardé: /content/mcp-brief-bot/examples/brief_robotique_médicale_2025-08-20_03-38.md

📊 Résumé généré avec 3 points et 3 sources

